In [3]:
# Just run this block. Please do not modify the following code.
import math
import time
import io
import numpy as np
import csv
from IPython.display import Image

# Pytorch package
import torch
import torch.nn as nn
import torch.optim as optim

# Torchtest package
import torchtext
from torchtext.datasets import Multi30k
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab
from torchtext.utils import download_from_url, extract_archive
from torch.nn.utils.rnn import pad_sequence

# Tqdm progress bar
from tqdm import tqdm_notebook, tqdm

# Code provide to you for training and evaluation

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

torch.seed

/opt/homebrew/anaconda3/envs/cs7643-finalproject/lib/python3.9/site-packages/torchtext/datasets/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/opt/homebrew/anaconda3/envs/cs7643-finalproject/lib/python3.9/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/opt/homebrew/anaconda3/envs/cs7643-finalproject/lib/python3.9/site-pack

In [70]:
# Check device availability
device = ''
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
device = torch.device(device)
print("You are using device: %s" % device)

You are using device: mps


In [71]:
encoder_output = torch.rand((5, 320, 20)).to(device) # N, input_size, 1

In [102]:
from Decoder import Decoder
from PostNet import PostNet

In [104]:
decoder = Decoder(lstm_bidirectional=True).to(device)
decoder_output = decoder.forward(encoder_output)
print(decoder_output.shape) # N, output_size, 1
print(decoder_output)

torch.Size([5, 80, 20])
tensor([[[-0.0074, -0.0202, -0.0404,  ..., -0.0122,  0.0116,  0.0369],
         [-0.0391, -0.0675, -0.0440,  ..., -0.0482, -0.0408, -0.0310],
         [-0.0471,  0.0003,  0.0272,  ..., -0.0253, -0.0217,  0.0309],
         ...,
         [-0.0411, -0.0500, -0.0412,  ..., -0.0170, -0.0244, -0.0423],
         [ 0.0167, -0.0110,  0.0521,  ...,  0.0336,  0.0598,  0.0603],
         [-0.0264, -0.0165, -0.0441,  ..., -0.0474, -0.0511,  0.0455]],

        [[-0.0199, -0.0015, -0.0085,  ..., -0.0073,  0.0356,  0.0287],
         [-0.0248, -0.0507, -0.0157,  ..., -0.0433, -0.0705, -0.0308],
         [-0.0311, -0.0459, -0.0448,  ...,  0.0181, -0.0088,  0.0313],
         ...,
         [-0.0298, -0.0323,  0.0026,  ..., -0.0607, -0.0520, -0.0403],
         [ 0.0357,  0.0347,  0.0352,  ..., -0.0047,  0.0305,  0.0717],
         [ 0.0133,  0.0022,  0.0095,  ...,  0.0068,  0.0019,  0.0380]],

        [[ 0.0125, -0.0015,  0.0147,  ...,  0.0093,  0.0306,  0.0324],
         [ 0.0223, -0

In [93]:
postnet = PostNet().to(device)
postnet_output = postnet.forward(decoder_output)
print(postnet_output.shape) # (N, output size, T)
print(postnet_output)

torch.Size([5, 80, 20])
tensor([[[ 0.4004,  0.3564,  0.0794,  ..., -0.2619, -0.1306, -0.1823],
         [-0.3228, -0.5483,  0.0567,  ...,  0.3977,  0.0078,  0.0659],
         [-0.1532, -0.3041, -0.4822,  ..., -0.1152, -0.3246, -0.6561],
         ...,
         [ 0.0990, -0.0151,  0.1652,  ..., -0.2176, -0.1653, -0.0551],
         [ 0.1219,  0.2633,  0.4579,  ..., -0.3859,  0.3868,  0.0475],
         [ 0.4893,  0.1749,  0.2055,  ...,  0.1346,  0.3933,  0.1470]],

        [[-0.2078,  0.1908,  0.0227,  ..., -0.3803,  0.1928, -0.0802],
         [-0.2858, -0.1520, -0.1230,  ...,  0.4037,  0.3054, -0.1552],
         [-0.0607,  0.5977, -0.0547,  ..., -0.0456, -0.3878, -0.3753],
         ...,
         [ 0.3754,  0.2060,  0.6203,  ...,  0.0237,  0.2181,  0.5058],
         [ 0.1301,  0.4488,  0.3396,  ..., -0.2713,  0.1827, -0.2259],
         [-0.0623,  0.2255, -0.1077,  ..., -0.3112, -0.2637,  0.0159]],

        [[ 0.4192,  0.3441,  0.5968,  ...,  0.1366, -0.0450,  0.0798],
         [ 0.1129, -0

In [75]:
# Loss Test
from utils import criterion

input = torch.rand((5, 80, 1)).to(device)

criterion(input=input, encoder_ouput=encoder_output, decoder_output=decoder_output, postnet_output=postnet_output, encoded_postnet_output=encoder_output)

/opt/homebrew/anaconda3/envs/cs7643-finalproject/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([5, 80, 20])) that is different to the input size (torch.Size([5, 80, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(0.8126, device='mps:0', grad_fn=<AddBackward0>)